# Import data

In [1]:
#imports
import os
import pandas as pd
import json
import ast
import re
from pprint import pprint
import numpy as np

In [2]:
#build df from database exctracted data
filename='Buildings_infos_2025-07-30.csv'
extraced_date=filename[-14:-4]
filepath = os.path.join('INs', filename)
df = pd.read_csv(filepath, 
    converters={
        'sub_components': ast.literal_eval,
        'sensors': ast.literal_eval,
        #'missing_WW_sensors': ast.literal_eval
    }
)

In [3]:
#inspest column
df.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'postal_code',
       'city', 'device_type', 'device_id', 'LayoutID', 'sub_components',
       'modular_system', 't_sensor_count', 'sensors', 'missing_WW_sensors',
       'gfid'],
      dtype='object')

# DF infos

In [4]:
#filter out uncesseray infos
df=df[['building_id','customerID','address','device_type','device_id','sensors', 'sub_components','missing_WW_sensors','gfid']]

In [5]:
#df.head()

In [6]:
df.count()

building_id           2209
customerID            2209
address               2209
device_type           1120
device_id             1120
sensors               2209
sub_components        2209
missing_WW_sensors     457
gfid                  1749
dtype: int64

## Display one building Detail

In [18]:
bid=2400 

In [19]:
df.loc[df['building_id'] == bid, 'sub_components'].item()

{'global(1)': 'id : 220711 category : NOT_SPECIFIED type : GLOBAL sub_type : NOT_SPECIFIED',
 'b(n)(1)': 'id : 220713 category : PRODUCER type : BOILER sub_type : GAS',
 'hp(n)(1)': 'id : 220712 category : PRODUCER type : HEAT_PUMP sub_type : NOT_SPECIFIED',
 'hc(n)(1)': 'id : 220714 category : CONSUMER type : HEATING_CIRCUIT sub_type : FLOOR_HEATING',
 'hw(n)(1)': 'id : 220715 category : CONSUMER type : WARM_WATER sub_type : STORAGE_TANK'}

In [20]:
df.loc[df['building_id'] == bid, 'missing_WW_sensors'].item()

"{'missing_gfids': [123], 'ww_subsystem_ids': [220715]}"

In [21]:
df.loc[df['building_id'] == bid, 'sensors'].item()

{'e_g_bldg_y(1)': 'id : 36519 gfid : 460 sub_system_id : 220711 unit : kWh source : MANUALLY_ENTERED',
 'None(1)': 'id : 40551 gfid : None sub_system_id : None unit : None source : GREENBOX_MQTT',
 't_hp(n)_secflow(1)': 'id : 40532 gfid : 369 sub_system_id : 220712 unit : °C source : GREENBOX_MQTT',
 't_hp(n)_secreturn(1)': 'id : 40534 gfid : 371 sub_system_id : 220712 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_tank(1)': 'id : 40536 gfid : 124 sub_system_id : 220715 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_pwh(1)': 'id : 40538 gfid : 119 sub_system_id : 220715 unit : °C source : GREENBOX_MQTT',
 't_hw(n)_circ(1)': 'id : 40540 gfid : 118 sub_system_id : 220715 unit : °C source : GREENBOX_MQTT',
 't_b(n)_flow(1)': 'id : 40542 gfid : 71 sub_system_id : 220713 unit : °C source : GREENBOX_MQTT',
 't_g_out_api(None)': 'id : 40552 gfid : 434 sub_system_id : None unit : °C source : OPEN_WEATHER_API',
 'None(None)': 'id : 40556 gfid : None sub_system_id : None unit : None source : GREENBO

In [22]:
df.loc[df['building_id'] == bid, ['device_id','customerID','address','gfid']]

,device_id,customerID,address,gfid
2042,2505022,178,Oscar-Kjellberg-Straße 3,OPS-22212


## Get rid of nan device_type and nan LayoutID (API buildings ???) 

In [23]:
type_of_devices=set()
for dev_type in df['device_type']:
    type_of_devices.add(dev_type)
type_of_devices   

{'ECR_LW300', 'RUT956', 'RevPiConnectSE', 'RevPiCore32SE', 'WAGOPFC200'}

In [37]:
df = df.dropna(subset=['device_type'])
# get rid of customer_id 35 (GF insternal)
df = df.loc[df['customerID'] != 35]

In [38]:
df.count()

building_id           1113
customerID            1113
address               1113
device_type           1113
device_id             1113
sensors               1113
sub_components        1113
missing_WW_sensors     105
gfid                  1113
fws_ids               1113
dtype: int64

In [36]:
df.loc[df['customerID'] != 35]

,building_id,customerID,address,device_type,device_id,sensors,sub_components,missing_WW_sensors,gfid,fws_ids
0,756,66,Wolzogenstr.28,RUT956,2403001,{'t_b(n)_flow(1)': 'id : 17772 gfid : 71 sub_s...,{'global(1)': 'id : 2204 category : NOT_SPECIF...,NaN,GFP-12505,[]
1,757,66,Hagelberger Str. 26,RUT956,2406009,{'e_g_bldg_y(None)': 'id : 19378 gfid : 460 su...,{'global(1)': 'id : 2209 category : NOT_SPECIF...,NaN,GFP-13438,[]
2,758,66,Hochstr. 8,RUT956,2420012,{'t_b(n)_flow(1)': 'id : 17862 gfid : 71 sub_s...,{'global(1)': 'id : 2213 category : NOT_SPECIF...,NaN,GFP-12475,[]
3,759,66,Planufer 82a,RUT956,2403011,{'t_b(n)_flow(1)': 'id : 17895 gfid : 71 sub_s...,{'global(1)': 'id : 2216 category : NOT_SPECIF...,NaN,GFP-12530,[]
4,761,66,Gabainstr. 13,RUT956,2440001,{'e_g_bldg_y(None)': 'id : 19747 gfid : 460 su...,{'global(1)': 'id : 2219 category : NOT_SPECIF...,NaN,GFP-12605,[]
...,...,...,...,...,...,...,...,...,...,...
2192,2300,167,Tulbeckstr. 11,RUT956,2508002,{'None(1)': 'id : 37021 gfid : None sub_system...,{'global(1)': 'id : 219998 category : NOT_SPEC...,NaN,OPS-176,[]
2193,2301,167,Sylvensteinstr. 7,RUT956,2508005,{'t_dh_primflow(1)': 'id : 37045 gfid : 88 sub...,{'global(1)': 'id : 219999 category : NOT_SPEC...,NaN,OPS-177,[]
2194,2302,167,Krüner Str. 2,RUT956,2511049,{'e_g_bldg_y(1)': 'id : 34296 gfid : 460 sub_s...,{'global(1)': 'id : 220000 category : NOT_SPEC...,NaN,OPS-178,[]
2195,2303,167,Oberländerstraße 5,RUT956,2507005,{'e_g_bldg_y(1)': 'id : 34297 gfid : 460 sub_s...,{'global(1)': 'id : 220001 category : NOT_SPEC...,NaN,OPS-179,[]


# Find FWS

In [26]:
def find_fws_ids(sub_components_dict):
    """
    Searches a sub_components dictionary for all FRESH_WATER_STATIONs.
    
    Returns:
        A list of all found component IDs. Returns an empty list if none are found.
    """
    # Return an empty list if the input isn't a dictionary
    if not isinstance(sub_components_dict, dict):
        return []
    
    found_ids = [] # Initialize an empty list to store IDs
    
    # Iterate through all component strings
    for component_string in sub_components_dict.values():
        if 'sub_type : FRESH_WATER_STATION' in component_string:
            try:
                parts = component_string.split()
                id_index = parts.index('id')
                component_id = int(parts[id_index + 2])
                # Add the found ID to the list instead of returning
                found_ids.append(component_id)
            except (ValueError, IndexError):
                # Malformed string, skip to the next one
                continue
                
    # After checking all components, return the list of found IDs
    return found_ids

In [27]:
# Apply the function to filter the DataFrame
df['fws_ids'] = df['sub_components'].apply(find_fws_ids)

In [28]:
df.loc[df['building_id'] == bid, 'fws_ids'].item()

[]

In [29]:
# Show the building_ids
buildings_with_fws = df.loc[df['fws_ids'].str.len() > 0, 'building_id'].tolist()
print(len(buildings_with_fws),buildings_with_fws)

56 [781, 782, 784, 785, 420, 442, 538, 562, 566, 434, 545, 1569, 792, 793, 797, 798, 799, 800, 801, 2453, 1474, 1475, 1476, 1477, 1478, 1481, 654, 392, 1401, 1404, 1405, 525, 532, 626, 700, 701, 370, 1605, 1606, 473, 474, 475, 476, 1364, 1457, 1461, 2250, 469, 472, 2431, 2470, 2382, 2383, 477, 2463, 2483]


In [30]:
buildings_with_fws=df.loc[df['fws_ids'].str.len() > 0]
buildings_with_fws.head()

,building_id,customerID,address,device_type,device_id,sensors,sub_components,missing_WW_sensors,gfid,fws_ids
23,781,66,Jahnstraße 14,RUT956,2420017,{'None(1)': 'id : 28260 gfid : None sub_system...,{'global(1)': 'id : 2282 category : NOT_SPECIF...,"{'missing_gfids': [124], 'ww_subsystem_ids': [...",GFP-13262,[2286]
24,782,66,Jahnstraße 30,RUT956,2420014,{'t_b(n)_flow(1)': 'id : 18129 gfid : 71 sub_s...,{'global(1)': 'id : 2288 category : NOT_SPECIF...,NaN,GFP-13285,"[2292, 2293]"
26,784,66,Jahnstraße 42,RUT956,2402007,{'None(1)': 'id : 18633 gfid : None sub_system...,{'global(1)': 'id : 2300 category : NOT_SPECIF...,NaN,GFP-13331,[2304]
27,785,66,Jahnstraße 46,RUT956,2403013,{'t_hw(n)_pwh(1)': 'id : 18118 gfid : 119 sub_...,{'global(1)': 'id : 2306 category : NOT_SPECIF...,NaN,GFP-13354,"[2310, 2311]"
34,420,32,Rendsburger Str. 61,ECR_LW300,23321007,{'t_hp(n)_secflow(1)': 'id : 7728 gfid : 369 s...,{'global(1)': 'id : 988 category : NOT_SPECIFI...,NaN,GFP-390,[991]


# get rid of FWS and missing_WW_sensors=Nan

In [31]:
buildings_without_fws = df.loc[df['fws_ids'].str.len() == 0]
buildings_without_fws=buildings_without_fws.dropna(subset=['missing_WW_sensors'])

In [32]:
buildings_without_fws.tail()

,building_id,customerID,address,device_type,device_id,sensors,sub_components,missing_WW_sensors,gfid,fws_ids
1991,2338,174,Horstmarer Landweg 74,RUT956,2507049,{'e_g_bldg_y(1)': 'id : 36067 gfid : 460 sub_s...,{'global(1)': 'id : 220296 category : NOT_SPEC...,"{'missing_gfids': [118, 119, 124], 'ww_subsyst...",OPS-21587,[]
2017,2384,177,Tivolistr. 106,RUT956,2507035,{'e_g_bldg_y(1)': 'id : 36493 gfid : 460 sub_s...,{'global(1)': 'id : 220404 category : NOT_SPEC...,"{'missing_gfids': [118], 'ww_subsystem_ids': [...",OPS-21987,[]
2042,2400,178,Oscar-Kjellberg-Straße 3,RUT956,2505022,{'e_g_bldg_y(1)': 'id : 36519 gfid : 460 sub_s...,{'global(1)': 'id : 220711 category : NOT_SPEC...,"{'missing_gfids': [123], 'ww_subsystem_ids': [...",OPS-22212,[]
2171,2461,187,Reichelsweiherstr. 9,RUT956,2505050,{'e_g_bldg_y(1)': 'id : 37616 gfid : 460 sub_s...,{'global(1)': 'id : 220835 category : NOT_SPEC...,"{'missing_gfids': [123, 124], 'ww_subsystem_id...",OPS-23006,[]
2191,2299,167,Garmischer Straße 201,RUT956,2508003,{'t_dh_primflow(1)': 'id : 37026 gfid : 88 sub...,{'global(1)': 'id : 219997 category : NOT_SPEC...,"{'missing_gfids': [118, 119, 123, 124], 'ww_su...",OPS-175,[]


# Export

In [33]:
if True:
    filename1='Building_infos_'+extraced_date+'.csv'
    filename2='Buildings_with_fws_'+extraced_date+'.csv'
    filename3='Buildings_nofws_missing_ww_sensors_'+extraced_date+'.csv'
    filepath1 = os.path.join('OUTs', filename1)
    filepath2 = os.path.join('OUTs', filename2)
    filepath3 = os.path.join('OUTs', filename3)
    df.to_csv(filepath1, index=False)
    buildings_with_fws.to_csv(filepath2, index=False)
    buildings_without_fws.to_csv(filepath3, index=False)
